<a href="https://colab.research.google.com/github/nafiuopeyemi88/ML_EDA_REPO/blob/main/Jordan_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geemap
!pip install geopandas
!pip install rasterstats
!pip install scipy
!pip install dask_geopandas

import ee
import geemap
import os
import math
import glob #reading and matching files
import numpy as np #array manipulation

import matplotlib.pyplot as plt

import pandas as pd
import geopandas as gpd

import dask_geopandas





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.0/55.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: dask
    Found existing installation: dask 2024.12.1
    Uninstalling dask-2024.12.1:
      Successfully uninstalled dask-2024.12.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2024.12.1 requires dask==2024.12.1, but you have dask 2025.4.0 which is incompatible.
dask-expr 1.1.21 requires dask==2024.12.1, but you have dask 2025.4.0 which is incompatible.
rapids-d

In [ ]:
#initiialize ee
ee.Authenticate()
ee.Initialize(project='ee-nafiuopeyemie')

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
jordan_gdf = gpd.read_file("/content/geoBoundaries-JOR-ADM0_simplified.geojson")
jordan_gdf.head()

,shapeName,shapeISO,shapeID,shapeGroup,shapeType,geometry
0,the Hashemite Kingdom of Jordan,JOR,64752131B76849546124065,JOR,ADM0,"POLYGON ((39.07694 32.33079, 39.06063 32.3343,..."


In [ ]:
jordan_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
global_df = pd.read_csv('sol_chem.pnts_horizons.csv', encoding='latin-1')
global_df.head(3)

<ipython-input-2-4559d09b7e71>:1: DtypeWarning: Columns (0,1,2,5,6,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  global_df = pd.read_csv('sol_chem.pnts_horizons.csv', encoding='latin-1')


,site_key,upedonid,site_obsdate,longitude_decimal_degrees,latitude_decimal_degrees,SSL_classification_name,labsampnum,layer_key,layer_sequence,hzn_top,...,ec_predict_one_to_two,source_db,confidence_degree,project_url,citation_url,hzn_depth,uuid,olc_id,dsiteid,layer_sequence.f
0,icr006475,Pandambili,2011.0,36.435982,-6.088750,NaN,Pan.2.1.Subsoil.Std fine soil,NaN,2.0,20.0,...,NaN,AfSIS1,2.0,https://registry.opendata.aws/afsis/,https://doi.org/10.1016/j.geodrs.2015.06.002,35.0,d5c40af3b570a290e8683dfb86d44932,6G5RWC6P+G95,AfSIS1_icr006475_2011,2.0
1,icr006586,Pandambili,2011.0,36.457722,-6.055750,NaN,Pan.8.1.Topsoil.Std fine soil,NaN,1.0,0.0,...,NaN,AfSIS1,2.0,https://registry.opendata.aws/afsis/,https://doi.org/10.1016/j.geodrs.2015.06.002,10.0,963d9362c24d0cddcada77a719ae9c0e,6G5RWFV5+P35,AfSIS1_icr006586_2011,1.0
2,icr007929,Kiberashi,2011.0,37.493351,-5.339337,NaN,Kib.11.1.Subsoil.Std fine soil,NaN,2.0,20.0,...,NaN,AfSIS1,2.0,https://registry.opendata.aws/afsis/,https://doi.org/10.1016/j.geodrs.2015.06.002,35.0,bb8d0173d837ee03dfc502fcb25e6166,6G6VMF6V+789,AfSIS1_icr007929_2011,2.0


In [ ]:
#create a geojson from the df, using longitude_decimal_degrees and latitude_decimal_degrees
global_gdf = gpd.GeoDataFrame(global_df, geometry=gpd.points_from_xy(global_df.longitude_decimal_degrees, global_df.latitude_decimal_degrees))


In [ ]:
if global_gdf.crs is None:
    global_gdf.set_crs("epsg:4326", inplace=True)

In [ ]:
#clip italy_gdf polgon out of the points
jordan_points = gpd.clip(global_gdf, jordan_gdf)
jordan_points.shape

(103, 45)

In [ ]:
jordan_points

,site_key,upedonid,site_obsdate,longitude_decimal_degrees,latitude_decimal_degrees,SSL_classification_name,labsampnum,layer_key,layer_sequence,hzn_top,hzn_bot,hzn_desgn,texture_description,texture_lab,clay_total,silt_total,sand_total,organic_carbon,oc_d,total_carbon_ncs,total_nitrogen_ncs,ph_kcl,ph_h2o,ph_cacl2,sum_of_cations_cec_pH_8_2,cec_nh4_ph_7,ecec_base_plus_aluminum,total_frag_wt_pct_gt_2_mm_ws,bulk_density_oven_dry,ca_ext,mg_ext,na_ext,k_ext,ec_water_extractable,ec_predict_one_to_two,source_db,confidence_degree,project_url,citation_url,hzn_depth,uuid,olc_id,dsiteid,layer_sequence.f,geometry
553509,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02698,64358.0,4.0,62.0,84.0,2Bym,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.46,NaN,NaN,NaN,NaN,NaN,NaN,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,73.0,761dd488db0439d0d7c27faf796ffa73,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,4.0,POINT (36.73333 31.76667)
553511,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02700,64360.0,6.0,100.0,144.0,2Bk2,Fine sandy loam,sl,17.4,15.0,67.6,5.20,2.820,0.61,0.29,NaN,7.80,7.9,NaN,10.6,NaN,11.0,0.61,15180.0,4080.0,19300.0,1420.0,NaN,31.10,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,122.0,c71543464d4ffadbd7bbd24d88bf1207,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,6.0,POINT (36.73333 31.76667)
553507,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02696,64356.0,2.0,2.0,27.0,2Bw1,Fine sandy loam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.22,NaN,NaN,NaN,NaN,NaN,NaN,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,14.5,a0735c9826bbce4989975a11fe35297b,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,2.0,POINT (36.73333 31.76667)
553508,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02697,64357.0,3.0,27.0,62.0,2Bw2,Fine sandy loam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.36,NaN,NaN,NaN,NaN,NaN,NaN,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,44.5,b8ec9599c839804c493d99c412f1b42d,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,3.0,POINT (36.73333 31.76667)
553506,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02695,64355.0,1.0,0.0,2.0,A,sil,l,16.4,41.7,41.9,6.11,7.130,3.55,0.49,NaN,7.80,7.8,NaN,12.0,NaN,19.0,1.44,NaN,371.0,184.0,485.0,NaN,2.53,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,1.0,3a4d042da3436a377823e72f073f83f4,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,1.0,POINT (36.73333 31.76667)
553510,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02699,64359.0,5.0,84.0,100.0,2Bk1,Fine sandy loam,lcos,10.0,9.2,80.8,2.21,0.839,0.24,0.00,NaN,7.90,7.9,NaN,12.8,NaN,27.0,0.52,NaN,1430.0,2210.0,766.0,NaN,6.78,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,92.0,085e7b0e3bfeba51dd51900c0fed80a5,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,5.0,POINT (36.73333 31.76667)
553512,9484,82FN500008,1982-08-01,36.733334,31.766666,"Coarse-loamy, gypsic, thermic Leptic Haplogypsid",82P02701,64361.0,7.0,144.0,170.0,3C,Fine sandy loam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.63,NaN,NaN,NaN,NaN,NaN,NaN,USDA_NCSS,1.0,http://ncsslabdatamart.sc.egov.usda.gov/,http://ncsslabdatamart.sc.egov.usda.gov/,157.0,8453ea93d7faf635b3b84459bdbf268f,8G3RQP8M+M88,USDA_NCSS_9484_1982-08-01,6.0,POINT (36.73333 31.76667)
553527,9487,82FN500011,1982-08-01,36.833332,31.833334,"Clayey-skeletal, mixed, thermic Aquic Torriort...",82P02716,64376.0,3.0,10.0,36.0,2Bz1,cl,c,56.6,19.0,24.4,2.60,1.540,NaN,0.11,NaN,9.00,9.0,NaN,15.1,NaN,57.0,1.

In [ ]:
jordan_points.to_file("jordan_profile.geojson", driver='GeoJSON')

In [ ]:
jordan_points.columns

Index(['site_key', 'upedonid', 'site_obsdate', 'longitude_decimal_degrees',
       'latitude_decimal_degrees', 'SSL_classification_name', 'labsampnum',
       'layer_key', 'layer_sequence', 'hzn_top', 'hzn_bot', 'hzn_desgn',
       'texture_description', 'texture_lab', 'clay_total', 'silt_total',
       'sand_total', 'organic_carbon', 'oc_d', 'total_carbon_ncs',
       'total_nitrogen_ncs', 'ph_kcl', 'ph_h2o', 'ph_cacl2',
       'sum_of_cations_cec_pH_8_2', 'cec_nh4_ph_7', 'ecec_base_plus_aluminum',
       'total_frag_wt_pct_gt_2_mm_ws', 'bulk_density_oven_dry', 'ca_ext',
       'mg_ext', 'na_ext', 'k_ext', 'ec_water_extractable',
       'ec_predict_one_to_two', 'source_db', 'confidence_degree',
       'project_url', 'citation_url', 'hzn_depth', 'uuid', 'olc_id', 'dsiteid',
       'layer_sequence.f', 'geometry'],
      dtype='object')

In [ ]:
#depth, #ph, #ec_1, #p_1, #k_1, ca1

In [ ]:
sample_points = gpd.read_file("/content/Soil_Analysis_2022.shp")
sample_points

,loc_nam,depth_1,pH_1,EC_1,SAR_1,P_1,K_1,Ca_1,Mg_1,Na_1,loc_nam_1,depth_2,pH_2,EC_2,SAR_2,P_2,K_2,Ca_2,Mg_2,Na_2,geometry
0,N7A,0-30,6.9,9.68,11.92,4.8,121.96,290.0,68.0,159.50,N7B,30-60,6.9,9.50,8.40,1.6,108.90,571.0,242.9,169.60,POINT Z (38.09531 32.48227 0)
1,N8A,0-30,7.1,11.07,20.64,4.6,121.96,195.0,8.5,208.18,N8B,30-60,7.0,13.00,28.10,2.1,79.20,288.8,183.4,432.40,POINT Z (38.09366 32.48602 0)
2,N1A,0-30,7.2,9.06,20.38,4.1,100.96,135.0,65.0,203.75,N1B,30-60,11.6,16.10,16.10,1.3,59.40,220.0,88.0,199.30,POINT Z (38.08501 32.47883 0)
3,N4A,0-30,8.0,23.10,3.77,11.4,321.46,14.0,1.9,10.63,N4B,30-60,7.6,1.70,7.80,1.3,247.50,9.8,23.2,31.50,POINT Z (38.08385 32.48292 0)
4,N3A,0-30,6.8,1.03,6.57,5.0,184.96,420.0,397.0,132.87,N3B,30-60,7.0,15.80,6.50,3.1,108.90,500.0,194.0,121.80,POINT Z (38.07521 32.47561 0)
5,N2A,0-30,8.0,0.51,3.87,3.9,79.96,5.0,4.0,8.20,N2B,30-60,7.9,0.70,5.30,3.1,277.20,1.8,2.1,7.40,POINT Z (38.07417 32.47979 0)
6,N5A,0-30,8.6,0.43,3.30,6.5,384.46,4.0,1.0,5.22,N5B,30-60,8.3,0.90,11.70,4.8,287.10,1.5,2.8,17.20,POINT Z (38.06512 32.47241 0)
7,N6A,0-30,7.7,10.03,17.08,3.6,69.46,281.0,29.0,212.60,N6B,30-60,6.9,14.50,11.60,2.8,59.40,460.1,164.6,205.40,POINT Z (38.06336 32.47644 0)
8,M7A,0-30,6.9,17.40,14.80,9.4,99.00,414.0,229.2,265.20,M7B,30-60,7.0,13.30,19.00,2.5,89.10,281.0,143.0,277.10,POINT Z (37.00671 31.29241 0)
9,M8A,0-30,7.1,14.50,22.80,2.2,108.90,62.0,209.3,265.20,M8B,30-60,7.0,12.70,15.30,2.8,59.40,175.5,225.8,217.40,POINT Z (37.00778 31.28789 0)


In [ ]:
# jordan_gdf.to_crs(epsg=32637, inplace=True)
# #EPSG:32637

In [ ]:
jordan_points = gpd.read_file("/content/jordan_profile.geojson")
jordan_points.shape

(103, 45)

In [ ]:
jordan_points['point_id'] = range(1, len(jordan_points) + 1)

In [ ]:
jordan_fc = jordan_points[['geometry', 'point_id']]

In [ ]:
def gdf_to_fc(gdf):
    features = []
    for _, row in gdf.iterrows():
        point = ee.Geometry.Point([row.geometry.x, row.geometry.y])
        feature = ee.Feature(point, row.drop('geometry').to_dict())
        features.append(feature)
    return ee.FeatureCollection(features)

fc = gdf_to_fc(jordan_fc)



In [ ]:

class DataProcessing:
  @staticmethod
  def aggregate_monthly_data(df):
      month_map = {
        1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr", 5: "May", 6: "Jun",
        7: "Jul", 8: "Aug", 9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
    }

      df["Month"] = df["Month"].map(month_map)
      pivoted_df = df.pivot(index='point_id', columns='Month')
      pivoted_df.columns = [f"{col[0].lower()}_{col[1].lower()}" for col in pivoted_df.columns]
      pivoted_df.reset_index(inplace=True)
      return pivoted_df
  @staticmethod
  def landsat_processing():
    start_year = 2019
    end_year = 2024
    L7_SR_T1 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
    L8_SR_T1 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

    def cloudMask(image):
        qa = image.select('QA_PIXEL')
        cloudShadowBitMask = 1 << 3
        cloudsBitMask = 1 << 5
        mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0))
        return image.updateMask(mask)

    def apply_scale_factors(image):
      optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
      return image.addBands(optical_bands, None, True)

    def renameBands(image):
        bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL']
        new_bands = ['B', 'G', 'R', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
        return image.select(bands).rename(new_bands)
    l7coll = L7_SR_T1.map(cloudMask).map(apply_scale_factors).map(renameBands)
    l8coll = L8_SR_T1.map(cloudMask).map(apply_scale_factors).map(renameBands)

    L78 = l8coll.merge(l7coll)


    # Monthly Processing Function
    def process_month(month):
        # Filter images for specific month and years
        monthly_images = L78.filter(ee.Filter.calendarRange(month, month, 'month')) \
                            .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                            .filterBounds(fc)  # `fc` is your FeatureCollection (points)

        # Apply percentile reducer across all images
        monthly_percentile = monthly_images.mean()

        # Extract values for each point
        reduced_fc = monthly_percentile.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )

        # Add month info
        return reduced_fc.map(lambda feature: feature.set('Month', month))
    monthly_results = ee.List.sequence(1, 12).map(process_month)
    final_results = ee.FeatureCollection(monthly_results).flatten()
    gee_data = final_results.getInfo()
    features = gee_data['features']
    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)
    df_landsat_bds = pd.DataFrame(data)
    df_landsat_bds.drop(["pixel_qa"], axis = 1, inplace = True)
    df_landsat_bds = DataProcessing.aggregate_monthly_data(df_landsat_bds)
    return df_landsat_bds



  @staticmethod
  def sentinel_processing():
    def mask_s2_clouds(image):
        qa = image.select('QA60')
        cloud_bit_mask = 1 << 10
        cirrus_bit_mask = 1 << 11
        mask = (
          qa.bitwiseAnd(cloud_bit_mask)
          .eq(0)
          .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
        )

        # Scale only reflectance bands
        scaled_image = image.select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12']).divide(10000)

        return image.addBands(scaled_image, overwrite=True).updateMask(mask)

    sentinel_dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

    selected_bands = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'QA60']
    sentinel_dataset = sentinel_dataset.select(selected_bands)

    # Apply cloud mask
    s2_data = sentinel_dataset.map(mask_s2_clouds)
    # Temporal Range & Percentile
    start_year = 2019
    end_year = 2024

    # Monthly Processing Function
    def process_month(month):
        # Filter images for specific month and years
        monthly_images = s2_data.filter(ee.Filter.calendarRange(month, month, 'month')) \
                            .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                            .filterBounds(fc)  # `fc` is your FeatureCollection (points)

        # Apply percentile reducer across all images
        monthly_percentile = monthly_images.mean()

        # Extract values for each point
        reduced_fc = monthly_percentile.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )

        # Add month info
        return reduced_fc.map(lambda feature: feature.set('Month', month))
    monthly_results = ee.List.sequence(1, 12).map(process_month)
    final_results = ee.FeatureCollection(monthly_results).flatten()
    gee_data = final_results.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)
    df_sentinel_bds = pd.DataFrame(data)
    df_sentinel_bds.drop(["QA60"], axis = 1, inplace = True)
    df_sentinel_bds = DataProcessing.aggregate_monthly_data(df_sentinel_bds)
    df_sentinel_bds.columns = ['sentinel_' + col if col != 'point_id' else col for col in df_sentinel_bds.columns]
    return df_sentinel_bds

  @staticmethod
  def modis_lst_night_mean_processing():
    modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Night_1km')

    def resample_to_30m(image):
        return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)
    start_year = 2019
    end_year = 2024

    def process_month(month):
        # Filter images for specific month and years
        monthly_images = modis_lst.filter(ee.Filter.calendarRange(month, month, 'month')) \
                            .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                            .filterBounds(fc)

        #resampled_image = monthly_images.map(resample_to_30m)
        monthly_mean = monthly_images.mean()
        resampled_image = resample_to_30m(monthly_mean)
        # Extract values for each point
        reduced_fc = resampled_image.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )
        return reduced_fc.map(lambda feature: feature.set('Month', month))

    monthly_results = ee.List.sequence(1, 12).map(process_month)
    final_results = ee.FeatureCollection(monthly_results).flatten()
    gee_data = final_results.getInfo()
    features = gee_data['features']
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)

    df_modis_lst_n_mean = pd.DataFrame(data)
    df_modis_lst_n_mean.rename(columns = {'mean': "modis_night_mean"},   inplace = True)
    df_modis_lst_n_mean = DataProcessing.aggregate_monthly_data(df_modis_lst_n_mean)
    return df_modis_lst_n_mean

  @staticmethod
  def modis_lst_night_std_processing():
    modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Night_1km')
    # Resample Function
    def resample_to_30m(image):
        return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)

    start_year = 2019
    end_year = 2024
    percentile = [5, 50, 95]

    def process_month(month):
        # Filter images for specific month and years
        monthly_images = modis_lst.filter(ee.Filter.calendarRange(month, month, 'month')) \
                            .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                            .filterBounds(fc)

        #resampled_image = monthly_images.map(resample_to_30m)
        monthly_std = monthly_images.reduce(ee.Reducer.stdDev())
        resampled_image = resample_to_30m(monthly_std)
        # Extract values for each point
        reduced_fc = resampled_image.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )
        return reduced_fc.map(lambda feature: feature.set('Month', month))
    monthly_results = ee.List.sequence(1, 12).map(process_month)
    final_results = ee.FeatureCollection(monthly_results).flatten()
    gee_data = final_results.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)
    df_modis_lst_n_std = pd.DataFrame(data)
    df_modis_lst_n_std.rename(columns = {'mean': "modis_night_std_mean"},   inplace = True)
    df_modis_lst_n_std = DataProcessing.aggregate_monthly_data(df_modis_lst_n_std)
    return df_modis_lst_n_std

  @staticmethod
  def modis_lst_day_processing():
    modis_lst = ee.ImageCollection("MODIS/061/MOD11A1").select('LST_Day_1km')

    # Resample Function
    def resample_to_30m(image):
        return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)
    start_year = 2019
    end_year = 2024

    def process_month(month):
        # Filter images for specific month and years
        monthly_images = modis_lst.filter(ee.Filter.calendarRange(month, month, 'month')) \
                            .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                            .filterBounds(fc)

        #resampled_image = monthly_images.map(resample_to_30m)
        monthly_stats = monthly_images.reduce(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs=True)
        ).rename(['LST_mean', 'LST_stdDev'])
        resampled_image = resample_to_30m(monthly_stats)
        # Extract values for each point
        reduced_fc = resampled_image.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )
        return reduced_fc.map(lambda feature: feature.set('Month', month))
    monthly_results = ee.List.sequence(1, 12).map(process_month)
    final_results = ee.FeatureCollection(monthly_results).flatten()
    gee_data = final_results.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)
    df_modis_lst_d = pd.DataFrame(data)
    df_modis_lst_d = DataProcessing.aggregate_monthly_data(df_modis_lst_d)
    df_modis_lst_d.columns = ['modis_day_' + col if col != 'point_id' else col for col in df_modis_lst_d.columns]
    return df_modis_lst_d

  @staticmethod
  def modis_bands():
    modis_bands = ee.ImageCollection('MODIS/061/MOD09A1')

    # Resample Function
    def resample_to_30m(image):
        return image.resample('bicubic').reproject(crs='EPSG:4326', scale=30)
    start_year = 2019
    end_year = 2024
    def process_month(month):
        # Filter images for specific month and years
        monthly_images = modis_bands.filter(ee.Filter.calendarRange(month, month, 'month')) \
                            .filter(ee.Filter.calendarRange(start_year, end_year, 'year')) \
                            .filterBounds(fc)

        #resampled_image = monthly_images.map(resample_to_30m)
        monthly_percentile = monthly_images.mean()
        resampled_image = resample_to_30m(monthly_percentile)
        # Extract values for each point
        reduced_fc = resampled_image.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )
        return reduced_fc.map(lambda feature: feature.set('Month', month))

    monthly_results = ee.List.sequence(1, 12).map(process_month)
    final_results = ee.FeatureCollection(monthly_results).flatten()
    gee_data = final_results.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        lon, lat = feature['geometry']['coordinates']
        props['longitude'] = lon
        props['latitude'] = lat
        data.append(props)

    df_modis_bds = pd.DataFrame(data)
    df_modis_bds.drop(["latitude", "longitude", "QA", "RelativeAzimuth", "SolarZenith", "StateQA", "ViewZenith", "DayOfYear"], axis = 1, inplace = True)
    df_modis_bds = DataProcessing.aggregate_monthly_data(df_modis_bds)
    return df_modis_bds

  @staticmethod
  def sm2rain_processing():
    sm2rain_dataset = ee.Image("OpenLandMap/CLM/CLM_PRECIPITATION_SM2RAIN_M/v01").select(['jan', 'feb', 'mar',
                                                                                      'apr', 'may', 'jun',
                                                                                      'jul', 'aug', 'sep',
                                                                                      'oct', 'nov', 'dec'])
    reduced_fc = sm2rain_dataset.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),
            scale=30
        )
    gee_data = reduced_fc.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)

    df_sm2rain = pd.DataFrame(data)
    df_sm2rain.columns = ['sm2rain_' + col if col != 'point_id' else col for col in df_sm2rain.columns]
    return df_sm2rain

  @staticmethod
  def merit_dem_processing():
    merit_dem = ee.Image("MERIT/Hydro/v1_0_1").select("hnd")
    reduced_fc = merit_dem.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )
    gee_data = reduced_fc.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)
    df_merit = pd.DataFrame(data)
    df_merit.rename(columns = {'mean': "merit_hand"},   inplace = True)
    return df_merit

  @staticmethod
  def merit_upstream_processing():
    merit_dem = ee.Image("MERIT/Hydro/v1_0_1").select("upa")
    reduced_fc = merit_dem.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),  # Use mean or other reducers as needed
            scale=30
        )
    gee_data = reduced_fc.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)

    df_merit_upstream = pd.DataFrame(data)
    df_merit_upstream.rename(columns = {'mean': "merit_upstream"},   inplace = True)
    return df_merit_upstream

  @staticmethod
  def srtm_processing():
    elevation_dataset = ee.Image("USGS/SRTMGL1_003").select('elevation')

    reduced_fc = elevation_dataset.reduceRegions(
            collection=fc,
            reducer=ee.Reducer.mean(),
            scale=30
        )


    gee_data = reduced_fc.getInfo()
    features = gee_data['features']

    # Creating a DataFrame
    data = []
    for feature in features:
        props = feature['properties']
        data.append(props)

    df_srtm = pd.DataFrame(data)
    df_srtm.rename(columns = {'mean': "srtm"},   inplace = True)
    return df_srtm




In [ ]:
landsat_bands = DataProcessing.landsat_processing()
sentinel_bands = DataProcessing.sentinel_processing()
modis_lst_night_mean = DataProcessing.modis_lst_night_mean_processing()
modis_lst_night_std = DataProcessing.modis_lst_night_std_processing()
modis_lst_day = DataProcessing.modis_lst_day_processing()
modis_bands = DataProcessing.modis_bands()
sm2rain = DataProcessing.sm2rain_processing()
merit_dem = DataProcessing.merit_dem_processing()
merit_upstream = DataProcessing.merit_upstream_processing()
srtm = DataProcessing.srtm_processing()
srtm

,srtm,point_id
0,524,1
1,524,2
2,524,3
3,524,4
4,524,5
...,...,...
98,-261,99
99,-261,100
100,-261,101
101,822,102


In [ ]:
#merge all the dfs on tile_id
dfs = [landsat_bands, sentinel_bands, modis_bands, modis_lst_day, modis_lst_night_mean, modis_lst_night_std, sm2rain, merit_upstream, merit_dem, srtm]
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, on='point_id', how='outer')


In [ ]:
merged_df

,point_id,b_apr,b_aug,b_dec,b_feb,b_jan,b_jul,b_jun,b_mar,b_may,b_nov,b_oct,b_sep,g_apr,g_aug,g_dec,g_feb,g_jan,g_jul,g_jun,g_mar,g_may,g_nov,g_oct,g_sep,nir_apr,nir_aug,nir_dec,nir_feb,nir_jan,nir_jul,nir_jun,nir_mar,nir_may,nir_nov,nir_oct,nir_sep,r_apr,r_aug,r_dec,r_feb,r_jan,r_jul,r_jun,r_mar,r_may,r_nov,r_oct,r_sep,swir1_apr,swir1_aug,swir1_dec,swir1_feb,swir1_jan,swir1_jul,swir1_jun,swir1_mar,swir1_may,swir1_nov,swir1_oct,swir1_sep,swir2_apr,swir2_aug,swir2_dec,swir2_feb,swir2_jan,swir2_jul,swir2_jun,swir2_mar,swir2_may,swir2_nov,swir2_oct,swir2_sep,sentinel_b11_apr,sentinel_b11_aug,sentinel_b11_dec,sentinel_b11_feb,sentinel_b11_jan,sentinel_b11_jul,sentinel_b11_jun,sentinel_b11_mar,sentinel_b11_may,sentinel_b11_nov,sentinel_b11_oct,sentinel_b11_sep,sentinel_b12_apr,sentinel_b12_aug,sentinel_b12_dec,sentinel_b12_feb,sentinel_b12_jan,sentinel_b12_jul,sentinel_b12_jun,sentinel_b12_mar,sentinel_b12_may,sentinel_b12_nov,sentinel_b12_oct,sentinel_b12_sep,sentinel_b2_apr,sentinel_b2_aug,sentinel_b2_dec,sentinel_b2_feb,sentinel_b2_jan,sentinel_b2_jul,sentinel_b2_jun,sentinel_b2_mar,sentinel_b2_may,sentinel_b2_nov,sentinel_b2_oct,sentinel_b2_sep,sentinel_b3_apr,sentinel_b3_aug,sentinel_b3_dec,sentinel_b3_feb,sentinel_b3_jan,sentinel_b3_jul,sentinel_b3_jun,sentinel_b3_mar,sentinel_b3_may,sentinel_b3_nov,sentinel_b3_oct,sentinel_b3_sep,sentinel_b4_apr,sentinel_b4_aug,sentinel_b4_dec,sentinel_b4_feb,sentinel_b4_jan,sentinel_b4_jul,sentinel_b4_jun,sentinel_b4_mar,sentinel_b4_may,sentinel_b4_nov,sentinel_b4_oct,sentinel_b4_sep,sentinel_b8_apr,sentinel_b8_aug,sentinel_b8_dec,sentinel_b8_feb,sentinel_b8_jan,sentinel_b8_jul,sentinel_b8_jun,sentinel_b8_mar,sentinel_b8_may,sentinel_b8_nov,sentinel_b8_oct,sentinel_b8_sep,sur_refl_b01_apr,sur_refl_b01_aug,sur_refl_b01_dec,sur_refl_b01_feb,sur_refl_b01_jan,sur_refl_b01_jul,sur_refl_b01_jun,sur_refl_b01_mar,sur_refl_b01_may,sur_refl_b01_nov,sur_refl_b01_oct,sur_refl_b01_sep,sur_refl_b02_apr,sur_refl_b02_aug,sur_refl_b02_dec,sur_refl_b02_feb,sur_refl_b02_jan,sur_refl_b02_jul,sur_refl_b02_jun,sur_refl_b02_mar,sur_refl_b02_may,sur_refl_b02_nov,sur_refl_b02_oct,sur_refl_b02_sep,sur_refl_b03_apr,sur_refl_b03_aug,sur_refl_b03_dec,sur_refl_b03_feb,sur_refl_b03_jan,sur_refl_b03_jul,sur_refl_b03_jun,sur_refl_b03_mar,sur_refl_b03_may,sur_refl_b03_nov,sur_refl_b03_oct,sur_refl_b03_sep,sur_refl_b04_apr,sur_refl_b04_aug,sur_refl_b04_dec,sur_refl_b04_feb,sur_refl_b04_jan,sur_refl_b04_jul,sur_refl_b04_jun,sur_refl_b04_mar,sur_refl_b04_may,sur_refl_b04_nov,sur_refl_b04_oct,sur_refl_b04_sep,sur_refl_b05_apr,sur_refl_b05_aug,sur_refl_b05_dec,sur_refl_b05_feb,sur_refl_b05_jan,sur_refl_b05_jul,sur_refl_b05_jun,sur_refl_b05_mar,sur_refl_b05_may,sur_refl_b05_nov,sur_refl_b05_oct,sur_refl_b05_sep,sur_refl_b06_apr,sur_refl_b06_aug,sur_refl_b06_dec,sur_refl_b06_feb,sur_refl_b06_jan,sur_refl_b06_jul,sur_refl_b06_jun,sur_refl_b06_mar,sur_refl_b06_may,sur_refl_b06_nov,sur_refl_b06_oct,sur_refl_b06_sep,sur_refl_b07_apr,sur_refl_b07_aug,sur_refl_b07_dec,sur_refl_b07_feb,sur_refl_b07_jan,sur_refl_b07_jul,sur_refl_b07_jun,sur_refl_b07_mar,sur_refl_b07_may,sur_refl_b07_nov,sur_refl_b07_oct,sur_refl_b07_sep,modis_day_lst_mean_apr,modis_day_lst_mean_aug,modis_day_lst_mean_dec,modis_day_lst_mean_feb,modis_day_lst_mean_jan,modis_day_lst_mean_jul,modis_day_lst_mean_jun,modis_day_lst_mean_mar,modis_day_lst_mean_may,modis_day_lst_mean_nov,modis_day_lst_mean_oct,modis_day_lst_mean_sep,modis_day_lst_stddev_apr,modis_day_lst_stddev_aug,modis_day_lst_stddev_dec,modis_day_lst_stddev_feb,modis_day_lst_stddev_jan,modis_day_lst_stddev_jul,modis_day_lst_stddev_jun,modis_day_lst_stddev_mar,modis_day_lst_stddev_may,modis_day_lst_stddev_nov,modis_day_lst_stddev_oct,modis_day_lst_stddev_sep,modis_night_mean_apr,modis_night_mean_aug,modis_night_mean_dec,modis_night_mean_feb,modis_night_mean_jan,modis_night_mean_jul,modis_night_mean_jun,modis_night_mean_mar,modis_night_mean_may,modis_night_mean_nov,modis_night_mean_oct,modis_night_mean_sep,modis_

In [ ]:
merged_df.to_csv('jordan_df.csv', index=False)

In [ ]:
jordan_points.columns

Index(['site_key', 'upedonid', 'site_obsdate', 'longitude_decimal_degrees',
       'latitude_decimal_degrees', 'SSL_classification_name', 'labsampnum',
       'layer_key', 'layer_sequence', 'hzn_top', 'hzn_bot', 'hzn_desgn',
       'texture_description', 'texture_lab', 'clay_total', 'silt_total',
       'sand_total', 'organic_carbon', 'oc_d', 'total_carbon_ncs',
       'total_nitrogen_ncs', 'ph_kcl', 'ph_h2o', 'ph_cacl2',
       'sum_of_cations_cec_pH_8_2', 'cec_nh4_ph_7', 'ecec_base_plus_aluminum',
       'total_frag_wt_pct_gt_2_mm_ws', 'bulk_density_oven_dry', 'ca_ext',
       'mg_ext', 'na_ext', 'k_ext', 'ec_water_extractable',
       'ec_predict_one_to_two', 'source_db', 'confidence_degree',
       'project_url', 'citation_url', 'hzn_depth', 'uuid', 'olc_id', 'dsiteid',
       'layer_sequence.f', 'geometry', 'point_id'],
      dtype='object')

In [ ]:
jordan_to_merge = jordan_points[['clay_total', 'silt_total',
       'sand_total', 'organic_carbon', 'oc_d', 'total_carbon_ncs',
       'total_nitrogen_ncs', 'ph_kcl', 'ph_h2o', 'ph_cacl2',
       'sum_of_cations_cec_pH_8_2', 'cec_nh4_ph_7', 'ecec_base_plus_aluminum',
       'total_frag_wt_pct_gt_2_mm_ws', 'bulk_density_oven_dry', 'ca_ext',
       'mg_ext', 'na_ext', 'k_ext', 'ec_water_extractable', 'hzn_depth', 'point_id' ]]

In [ ]:
final_merge = merged_df.merge(jordan_to_merge, on='point_id', how='outer')

In [ ]:
final_merge.shape

(103, 313)

In [ ]:
final_merge.to_csv('final_merge_jordan.csv', index=False)

In [ ]:
gdf1 = gpd.read_file('bio1.geojson')
gdf2 = gpd.read_file('bio2.geojson')
gdf3 = gpd.read_file('bio3.geojson')
gdf4 = gpd.read_file('bio4.geojson')
gdf5 = gpd.read_file('bio5.geojson')
gdf6 = gpd.read_file('bio6.geojson')
gdf7 = gpd.read_file('bio7.geojson')
gdf8 = gpd.read_file('bio8.geojson')
gdf9 = gpd.read_file('bio9.geojson')
gdf10 = gpd.read_file('bio10.geojson')
gdf11 = gpd.read_file('bio11.geojson')
gdf12 = gpd.read_file('bio12.geojson')
gdf13 = gpd.read_file('bio13.geojson')
gdf14 = gpd.read_file('bio14.geojson')
gdf15 = gpd.read_file('bio15.geojson')
gdf16 = gpd.read_file('bio16.geojson')
gdf17 = gpd.read_file('bio17.geojson')
gdf18 = gpd.read_file('bio18.geojson')
gdf19 = gpd.read_file('bio19.geojson')

In [ ]:
gdf19.drop(['site_key', 'upedonid', 'site_obsdate', 'longitude_decimal_degrees',
       'latitude_decimal_degrees', 'SSL_classification_name', 'labsampnum',
       'layer_key', 'layer_sequence', 'hzn_top', 'hzn_bot', 'hzn_desgn',
       'texture_description', 'texture_lab', 'clay_total', 'silt_total',
       'sand_total', 'organic_carbon', 'oc_d', 'total_carbon_ncs',
       'total_nitrogen_ncs', 'ph_kcl', 'ph_h2o', 'ph_cacl2',
       'sum_of_cations_cec_pH_8_2', 'cec_nh4_ph_7', 'ecec_base_plus_aluminum',
       'total_frag_wt_pct_gt_2_mm_ws', 'bulk_density_oven_dry', 'ca_ext',
       'mg_ext', 'na_ext', 'k_ext', 'ec_water_extractable',
       'ec_predict_one_to_two', 'source_db', 'confidence_degree',
       'project_url', 'citation_url', 'hzn_depth', 'uuid', 'olc_id', 'dsiteid',
       'layer_sequence.f'], axis = 1, inplace = True)

In [ ]:
gdf9.columns

Index(['bio9', 'geometry'], dtype='object')

In [ ]:

def add_point_id_to_df(df):
  df['point_id'] = range(1, len(df) + 1)
  return df

gdf1 = add_point_id_to_df(gdf1)
gdf2 = add_point_id_to_df(gdf2)
gdf3 = add_point_id_to_df(gdf3)
gdf4 = add_point_id_to_df(gdf4)
gdf5 = add_point_id_to_df(gdf5)
gdf6 = add_point_id_to_df(gdf6)
gdf7 = add_point_id_to_df(gdf7)
gdf8 = add_point_id_to_df(gdf8)
gdf9 = add_point_id_to_df(gdf9)
gdf10 = add_point_id_to_df(gdf10)
gdf11 = add_point_id_to_df(gdf11)
gdf12 = add_point_id_to_df(gdf12)
gdf13 = add_point_id_to_df(gdf13)
gdf14 = add_point_id_to_df(gdf14)
gdf15 = add_point_id_to_df(gdf15)
gdf16 = add_point_id_to_df(gdf16)
gdf17 = add_point_id_to_df(gdf17)
gdf18 = add_point_id_to_df(gdf18)
gdf19 = add_point_id_to_df(gdf19)


In [ ]:
#drop geometry
def drop_geometry(df):
  df.drop(['geometry'], axis = 1, inplace = True)
  return df

gdf1 = drop_geometry(gdf1)
gdf2 = drop_geometry(gdf2)
gdf3 = drop_geometry(gdf3)
gdf4 = drop_geometry(gdf4)
gdf5 = drop_geometry(gdf5)
gdf6 = drop_geometry(gdf6)
gdf7 = drop_geometry(gdf7)
gdf8 = drop_geometry(gdf8)
gdf9 = drop_geometry(gdf9)
gdf10 = drop_geometry(gdf10)
gdf11 = drop_geometry(gdf11)
gdf12 = drop_geometry(gdf12)
gdf13 = drop_geometry(gdf13)
gdf14 = drop_geometry(gdf14)
gdf15 = drop_geometry(gdf15)
gdf16 = drop_geometry(gdf16)
gdf17 = drop_geometry(gdf17)
gdf18 = drop_geometry(gdf18)
gdf19 = drop_geometry(gdf19)


In [ ]:
#merge all to a single
#merge all the dfs on tile_id
gdfs = [gdf1, gdf2, gdf3, gdf4, gdf5, gdf7, gdf8, gdf10, gdf12, gdf13, gdf14, gdf15, gdf16, gdf17, gdf18, gdf19]
merged_df = gdfs[0]
for df in gdfs[1:]:
    merged_df = merged_df.merge(df, on='point_id', how='outer')


In [ ]:
merged_df

,bio1,point_id,bio2,bio3,bio4,bio5,bio7,bio8,bio10,bio12,bio13,bio14,bio15,bio16,bio17,bio18,bio19
0,19.75,1,13.6,0.414,722.0,36.15,32.8,10.05,28.25,63.0,15.7,0.0,96.1,37.2,0.1,0.1,37.2
1,19.75,2,13.6,0.414,722.0,36.15,32.8,10.05,28.25,63.0,15.7,0.0,96.1,37.2,0.1,0.1,37.2
2,19.75,3,13.6,0.414,722.0,36.15,32.8,10.05,28.25,63.0,15.7,0.0,96.1,37.2,0.1,0.1,37.2
3,19.75,4,13.6,0.414,722.0,36.15,32.8,10.05,28.25,63.0,15.7,0.0,96.1,37.2,0.1,0.1,37.2
4,19.75,5,13.6,0.414,722.0,36.15,32.8,10.05,28.25,63.0,15.7,0.0,96.1,37.2,0.1,0.1,37.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,22.25,99,12.3,0.438,579.3,36.65,28.1,14.45,28.95,468.9,112.4,0.5,108.3,306.6,2.0,2.2,306.6
99,22.25,100,12.3,0.438,579.3,36.65,28.1,14.45,28.95,468.9,112.4,0.5,108.3,306.6,2.0,2.2,306.6
100,22.25,101,12.3,0.438,579.3,36.65,28.1,14.45,28.95,468.9,112.4,0.5,108.3,306.6,2.0,2.2,306.6
101,16.55,102,12.1,0.414,640.5,31.75,29.1,8.05,24.15,632.1,149.0,0.8,106.5,404.5,3.1,3.1,404.5


In [ ]:
df = pd.read_csv('final_merge_jordan.csv')

final_merge = df.merge(merged_df, on='point_id', how='outer')

In [ ]:
final_merge.to_csv('JORDAN_TRAINING.csv', index=False)

In [ ]:
import geopandas as gpd
from shapely.geometry import box, Point
import numpy as np

def create_grid_and_centers(gdf, cell_size=30):

    bounds = gdf.total_bounds  # minx, miny, maxx, maxy
    minx, miny, maxx, maxy = bounds

    # Create grid cells
    x_coords = np.arange(minx, maxx, cell_size)
    y_coords = np.arange(miny, maxy, cell_size)

    grid_cells = []
    center_points = []

    for x in x_coords:
        for y in y_coords:
            cell = box(x, y, x + cell_size, y + cell_size)
            if gdf.intersects(cell).any():
                grid_cells.append(cell)
                center = Point(x + cell_size / 2, y + cell_size / 2)
                center_points.append(center)

    grid_gdf = gpd.GeoDataFrame(geometry=grid_cells, crs=gdf.crs)

    return grid_gdf